In [40]:
import pandas as pd
import ast


In [41]:
#Functions 

def calculate_combinations(row):
    result = 1
    row = ast.literal_eval(row)
    for list in row:
        result = result * len(list)
    return result

In [42]:
# group categories by dyad_id and convert the global_category column to a list of lists

df = pd.read_csv('categories.csv')

df = pd.read_csv('categories.csv')

df['global_category'] = df['global_category'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
df = df[df['global_category'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

grouped_df = df.groupby('dyad_id')['global_category'].apply(list).reset_index()

grouped_df.to_csv('grouped_categories.csv', index=False)

In [ ]:
# Calculate the number of combinations for each dyad_id and sum them up

grouped_df = pd.read_csv('grouped_categories.csv')
grouped_df['num_combinations'] = grouped_df['global_category'].apply(calculate_combinations)
total_combinations = grouped_df['num_combinations'].sum()
print("Total combinations:", total_combinations)



Rows with num_combinations = 1: 161
Rows with num_combinations > 1: 339


In [50]:


df = pd.read_csv("grouped_categories.csv")

# This will hold lines in SPMF format

spmf_lines = []

for _, row in df.iterrows():
    dyad_id = row["dyad_id"]
    try:
        sequence = ast.literal_eval(row["global_category"])  # Parse the list of steps
        spmf_sequence = []
        for step in sequence:
            if isinstance(step, list) and step:
                items = [str(item) for item in step]
                spmf_sequence.append(" ".join(items) + " -1")
        spmf_sequence.append("-2")
        line = f"{dyad_id}: " + " ".join(spmf_sequence)
        spmf_lines.append(line)
    except Exception as e:
        print(f"❌ Error parsing row {dyad_id}: {e}")


    # Save the SPMF formatted lines to a CSV file
    with open("grouped_categories_spmf.csv", "w") as file:
        for line in spmf_lines:
            file.write(line + "\n")